# Importing preprocessed data set from .csv

In [1]:
# Importing modules
import pandas as pd

df = pd.read_csv('outData_l_NPE.csv', sep=";")
df


,Unnamed: 0,doc_id,text
0,1,38655005,Rätsel Epidemie China WHO Coronavirus Krankhei...
1,2,38656816,Sars-Trauma Medizin China Angst Erreger Lungen...
2,3,38672031,Angst Virus Lungenerkrankung China Bevölkerung...
3,4,38708579,Coronavirus Japan Tokio Japan Coronavirus Chin...
4,5,38726997,Coronavirus Forscher Infizierte London Ausbrei...
...,...,...,...
18707,18708,41624420,Tigers-Sieg Krisen-Derby SCB Ende ANGELO ROCCH...
18708,18709,41624422,Lions Monat Lindbohm ZSC-Serie Monat ZSC Lions...
18709,18710,41628255,Sportlerin Jahr Viola Amherd Vereinssportlerin...
18710,18711,41628256,Art Abschied Friede Michael Llamas Chefarzt Sp...


# Gensim Preprocessing
- Tokenization
- All lowercase
- Filter for single letter tokens
+ Adapted filter for longer tokens (tokens of 130 characters allowed)

In [2]:
import gensim
from gensim.utils import simple_preprocess

def text_to_words(texts):
    for text in texts:
        yield(gensim.utils.simple_preprocess(str(text), max_len=130))

data = df.text.values.tolist()
data_words = list(text_to_words(data))

print(data_words[:1][0][:30])

['rätsel', 'epidemie', 'china', 'who', 'coronavirus', 'krankheit', 'virus', 'sars', 'erreger', 'alan', 'niederer', 'ausbreitung', 'lungenentzündung', 'metropole', 'wuhan', 'coronavirus', 'weltgesundheitsorganisation', 'who', 'agenturberichten', 'donnerstag', 'peking', 'staatsfernsehen', 'einschätzung', 'test', 'experte', 'schluss', 'krankheit', 'typ', 'virusfamilie', 'wissenschafter']


Creating dictionary and token/Document matrices

In [3]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 3), (19, 4), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1)]


# Coherence Calculation
Preparation of additional function for coherence calculation on models

In [4]:
from gensim.models import CoherenceModel

def compute_coherence_values(corpus, dictionary, k, alpha, beta):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha = alpha,
                                           eta=beta)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

# Hyperparameter Training Loop
Currently only iterates a number of topics with no optimization for other hyperparameters

In [9]:
#### Topic optimizer
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}# Topics range
min_topics = 25
max_topics = 101
step_size = 25
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.98))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.98))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)

corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5),
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75),
               corpus]
corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Coherence': [],
                 'Alpha':[],
                 'Beta':[]
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=48)
    
    #iterate alpha values
    for a in alpha:
        #iterate beta values
        for b in beta:
            # iterate through validation corpuses
            for i in range(len(corpus_sets)):
                # iterate through number of topics
                for k in topics_range:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, k=k, alpha=a, beta=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Coherence'].append(cv)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)

                    pbar.update(1)
                    pbar.refresh()
    pd.DataFrame(model_results).to_csv('lda_tuning_results_l_NPE_hparams.csv', index=False)
    pbar.close()


  0%|          | 0/80 [17:03<?, ?it/s]
































































































100%|██████████| 48/48 [6:27:42<00:00, 484.64s/it]


# Training and visualization
Training specific model and visualizing results with pyLDAvis

In [5]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

num_topics = 50
num_of_docs = len(corpus)

lda_model = gensim.models.LdaMulticore(corpus=corpus, 
                                            num_topics=num_topics, 
                                            workers = 5,
                                            id2word=id2word,
                                            random_state=100,
                                            passes=10,
                                            minimum_probability = 0.0,
                                            alpha = 'asymmetric',
                                            eta = 'symmetric')
    # Visualize the topics
pyLDAvis.enable_notebook()
p = gensimvis.prepare(lda_model, corpus, id2word)

pyLDAvis.save_html(p, 'ldavis_tuned_lemma_NPE_'+ str(num_topics) +'_optim_umass.html')

C:\Users\Daniel\AppData\Local\Programs\Python\Python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


# Topic per Document extraction
Loop for Topic extraction per document

In [13]:
with open("TPD_L_NPE_50_optim.txt", "a") as file_object:
    test = list()
    
    for x in range(18712):
        test.clear()
        test.append(df["doc_id"][x])
        for i in range(num_topics):
            temp = lda_model[corpus[x]]
            test.append(temp[i][1])
        file_object.write(' '.join(map(str, test)))
        file_object.write("\n")


# Testcode zur Problematik der Themeannotation

In [107]:
#Testtext Tagesanzeiger https://www.tagesanzeiger.ch/anzeichen-fuer-langfristigen-immunschutz-gegen-corona-entdeckt-571881016955
teststring = "Zürcher Forschende berichten von einer molekularen Signatur, mit der sich der langfristige Immunschutz gegen das Coronavirus bereits während der akuten Infektionsphase abschätzen lässt. Im Fokus stehen sogenannte Gedächtnis-T-Zellen. Antikörper machen nur einen Teil des Waffenarsenals des Immunsystems aus. Ebenso wichtig sind die T-Zellen: Sie bekämpfen nicht direkt das Virus, sondern erkennen infizierte Zellen und zerstören diese. Ist das Virus besiegt, sterben die allermeisten dieser sogenannten Killerzellen wieder ab. Nur die wenigsten überleben und reifen zu langlebigen Gedächtnis-T-Zellen heran. Bei einer erneuten Infektion können sie das Virus schnell und wirksam bekämpfen. Forschenden um den Immunologen Onur Boyman, Direktor der Klinik für Immunologie am Universitätsspital Zürich, ist es nun gelungen, Sars-CoV-2-spezifische T-Zellen des Typs «CD8+» von der akuten Covid-19-Infektion bis zu einem Jahr nach Genesung in Blutproben von 175 Personen zu untersuchen. Von den Ergebnissen berichten sie im Fachmagazin «Nature». Demnach konnten sie eine eindeutige molekulare Signatur identifizieren, die es den T-Zellen erlauben, langlebige Gedächtniszellen zu werden und nicht nach Abklingen der akuten Infektion abzusterben. Die Signatur sei bereits während der akuten Infektion nachweisbar gewesen, teilte die Universität Zürich am Mittwoch mit. Schutz vor neuen Varianten Die Erkenntnisse der Studie könnten möglicherweise dazu dienen, gezielt einen langanhaltenden Schutz nach Impfung oder Infektion vorauszusagen: «Falls sich die nun identifizierte molekulare Signatur während einer Infektion nicht feststellen lässt, könnte man nach Abklingen der Krankheit impfen», erklärte Boyman gegenüber der Nachrichtenagentur Keystone-SDA. Und falls die Signatur nach einer Impfung nicht auftrete, könnten etwa mehr Impfdosen verabreicht oder die Inhaltsstoffe der Vakzine angepasst werden – «bis wir die molekulare Signatur sehen». Anzunehmen sei auch, dass die Bildung dieser Gedächtnis-T-Zellen dafür verantwortlich sei, dass die derzeit verfügbaren Impfstoffe auch gegen Omikron einen hohen Schutz vor Hospitalisierung und Tod bieten. «Denn für CD8+-T-Zellen spielen einzelne Mutationen auf dem Spikeprotein des Coronavirus eine geringere Rolle als für Antikörper», so der Immunologe." 
teststring = teststring.lower()
line = teststring.split()
line_bow = id2word.doc2bow(line)
print(line)
print(line_bow)

['zürcher', 'forschende', 'berichten', 'von', 'einer', 'molekularen', 'signatur,', 'mit', 'der', 'sich', 'der', 'langfristige', 'immunschutz', 'gegen', 'das', 'coronavirus', 'bereits', 'während', 'der', 'akuten', 'infektionsphase', 'abschätzen', 'lässt.', 'im', 'fokus', 'stehen', 'sogenannte', 'gedächtnis-t-zellen.', 'antikörper', 'machen', 'nur', 'einen', 'teil', 'des', 'waffenarsenals', 'des', 'immunsystems', 'aus.', 'ebenso', 'wichtig', 'sind', 'die', 't-zellen:', 'sie', 'bekämpfen', 'nicht', 'direkt', 'das', 'virus,', 'sondern', 'erkennen', 'infizierte', 'zellen', 'und', 'zerstören', 'diese.', 'ist', 'das', 'virus', 'besiegt,', 'sterben', 'die', 'allermeisten', 'dieser', 'sogenannten', 'killerzellen', 'wieder', 'ab.', 'nur', 'die', 'wenigsten', 'überleben', 'und', 'reifen', 'zu', 'langlebigen', 'gedächtnis-t-zellen', 'heran.', 'bei', 'einer', 'erneuten', 'infektion', 'können', 'sie', 'das', 'virus', 'schnell', 'und', 'wirksam', 'bekämpfen.', 'forschenden', 'um', 'den', 'immunologen

In [108]:
doc_lda = lda_model[line_bow]
print(doc_lda)



[(0, 0.11357095), (1, 0.035790242), (2, 0.0002602398), (3, 0.0243006), (4, 0.00021322719), (5, 0.00019556288), (6, 0.100728504), (7, 0.021211443), (8, 0.00015663473), (9, 0.00014688836), (10, 0.00013828384), (11, 0.00013063161), (12, 0.062765576), (13, 0.014707493), (14, 0.000112032896), (15, 0.00010695688), (16, 0.019842349), (17, 0.024312628), (18, 9.415844e-05), (19, 9.054683e-05), (20, 0.014007884), (21, 8.4095576e-05), (22, 8.120282e-05), (23, 0.019006822), (24, 0.046774674), (25, 7.360693e-05), (26, 7.138121e-05), (27, 0.017568346), (28, 6.7310546e-05), (29, 6.5444496e-05), (30, 6.3679116e-05), (31, 6.2006475e-05), (32, 0.023517467), (33, 0.33639297), (34, 5.7477264e-05), (35, 0.06906124), (36, 5.480832e-05), (37, 5.3564683e-05), (38, 5.2376232e-05), (39, 5.1239374e-05), (40, 5.015082e-05), (41, 4.910756e-05), (42, 4.8106816e-05), (43, 0.02531913), (44, 0.02827253), (45, 4.533521e-05), (46, 4.4480974e-05), (47, 4.3658336e-05), (48, 4.2865573e-05), (49, 4.2101085e-05)]


In [109]:
def max_value(inputlist):
    return max([sublist[-1] for sublist in inputlist])

print(max_value(doc_lda))

0.33639297


Functions for saving and loading trained models

In [6]:
lda_model.save("StoredModel")

In [ ]:
lda_model.load("StoredModel")